In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        if "train.csv" in filename or "test.csv" in filename:
            print(os.path.join(dirname, filename))

In [ ]:
import warnings
warnings.filterwarnings(action='ignore') # once

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
%matplotlib inline

In [ ]:
# # Function
# def set_plt_params(figsize, font, lw, color, grid=False):
#     plt.rcParams.update({'font.size':font, 
#                          'figure.figsize':figsize,
#                          'lines.linewidth':lw,
#                          'lines.color':color,
#                          'axes.grid':grid,
#                          'axes.unicode_minus':False
# #                          'cmap':cmap
#                         })
#     plt.style.use('bmh')
# #     font_location = 'c:/Windows/Fonts/malgun.ttf'
# #     font_name = font_manager.FontProperties(fname=font_location).get_name()
# #     rc('font', family=font_name)
    
# def initilization(): # 추후 생성자로 넣기
#     pd.set_option('display.float_format', 
#                   lambda x: '%.0f' % x if (x == x and x*10 % 10 == 0) 
#                   else ('%.1f' % x if (x == x and x*100 % 10 == 0)
#                         else '%.2f' % x))
#     set_plt_params((15,6),12,1,'b',True)
    
# initilization()

### Data Load
* Data Fields
>   datetime - hourly date + timestamp  
>   season -  1 = spring, 2 = summer, 3 = fall, 4 = winter     
>   holiday - whether the day is considered a holiday    
>   workingday - whether the day is neither a weekend nor holiday    
>   weather - 1: Clear, Few clouds, Partly cloudy, Partly cloudy      
>   2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist     
>   3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds     
>   4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog     
>   temp - temperature in Celsius    
>   atemp - "feels like" temperature in Celsius    
>   humidity - relative humidity    
>   windspeed - wind speed    
>   casual - number of non-registered user rentals initiated    
>   registered - number of registered user rentals initiated    
>   count - number of total rentals    

In [ ]:
# df.melt(id_vars='A', value_vars='B', var_name='C', value_name='D')
# A열을 기준으로 B열을 행으로 melt하면서, B컬럼명을 값으로 가지는 열이름=C, B컬럼이 원래 가졌던 값에 대한 열이름은D
# id_vars = 변하지 않는 컬럼, 반복될 수 있음
# value_vars = melting되는 값. 여러개면 C아래로 컬럼명 생성되는것

In [ ]:
df_train = pd.read_csv("/kaggle/input/bike-sharing-demand/train.csv")
df_test = pd.read_csv("/kaggle/input/bike-sharing-demand/test.csv")
display(df_train.head())
display(df_test.head())

In [ ]:
display(df_train.info())
display(df_test.info())

In [ ]:
# 시간 관련 변수 새로 생성
df_train['datetime'] = pd.to_datetime(df_train['datetime'])
df_train['yyyymm'] = df_train['datetime'].dt.to_period('M')
df_train['yyyymmdd'] = df_train['datetime'].dt.to_period('D')
df_train['hour'] = df_train['datetime'].dt.hour
df_train['year'] = df_train['datetime'].dt.year
df_train['month'] = df_train['datetime'].dt.month

df_test['datetime'] = pd.to_datetime(df_test['datetime'])
df_test['yyyymm'] = df_test['datetime'].dt.to_period('M')
df_test['yyyymmdd'] = df_test['datetime'].dt.to_period('D')
df_test['hour'] = df_test['datetime'].dt.hour
df_test['year'] = df_test['datetime'].dt.year
df_test['month'] = df_test['datetime'].dt.month

In [ ]:
df = df_train.copy(deep=True) # 편의를 위해 df로 변경

In [ ]:
df.describe()

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(20,5))
df['count'].plot(kind='hist', ax=axes[0])
df['count'].plot(ax=axes[1])

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(20,5))
df['temp'].plot(kind='hist', ax=axes[0])
df['atemp'].plot(kind='hist', ax=axes[1])

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(20,5))
df['humidity'].plot(kind='hist', ax=axes[0])
df['windspeed'].plot(kind='hist', ax=axes[1])

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(20,5))
df_test['humidity'].plot(kind='hist', ax=axes[0])
df_test['windspeed'].plot(kind='hist', ax=axes[1])

In [ ]:
## windspeed -> outlier detection
def outlier_detection(data, col_nm, rule=1.5):
    q1 = data[col_nm].quantile(0.25)
    q3 = data[col_nm].quantile(0.75)
    iqr = q3-q1
    condition = (data[col_nm] > q3 + rule*iqr) | (data[col_nm] < q1 - rule*iqr)
    print(condition.value_counts())
    return data.drop(data[condition].index)
df_od = outlier_detection(df, 'windspeed')
df_od

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(10,5))
df['windspeed'].plot(kind='box', ax=axes[0])
df_od['windspeed'].plot(kind='box', ax=axes[1])
plt.show()
fig, axes = plt.subplots(1,2, figsize=(10,5))
df['windspeed'].plot(kind='hist', ax=axes[0])
df_od['windspeed'].plot(kind='hist', ax=axes[1])

In [ ]:
print(len(df))
df = df_od
print(len(df))

In [ ]:
df_month_sum = df.groupby('yyyymm').sum().reset_index()
df_day_sum = df.groupby('yyyymmdd').sum().reset_index()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(df_day_sum['count'], model='additive', period=30)#, freq=30)  
result.plot() # 일별 수요에 대한 것은 큰 의미를 찾기가 힘듬 따라서 월별로 변경
plt.show()
result = seasonal_decompose(df_month_sum['count'], model='additive', period=12)#, freq=12)  # lag단위로 쪼갬(현재 )
result.plot()
plt.show()
# 자전거 공유서비스에 대한 전체 월별 수요는 증가하는 경향을 보이며, 계절성 존재 (겨울철에는 수요 낮고 봄부터 여름까지 상승)

In [ ]:
df_train.groupby(['yyyymm','season']).sum()

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(20,5))
df.groupby('season').sum()[['count']].plot(kind='bar', ax=axes[0])
df.groupby(['year','month']).sum().unstack()['count'].plot(kind='bar', ax=axes[1])
# 계절성 - 여름 가을에 수요가 높다는거
# 1 = spring, 2 = summer, 3 = fall, 4 = winter
# 2011년에 비해 2012년 수요가 증가했으니까 year, month도 feature로 추가

In [ ]:
df.groupby(['season']).mean()[['temp','atemp']].plot(kind='bar')
# 일반적으로 실제 온도 보다는 사람들이 체감하는 온도가 더 높다(심지어 겨울철에도)

In [ ]:
df.groupby(['hour']).sum()['count'].plot(kind='bar')
# 새벽시간대 이용률이 매우 낮고 출근시간 8시, 퇴근시간 17-18시에 이용률이 높다

In [ ]:
fig, axes = plt.subplots(1,2) # or ax=fig.add_subplot(221)
df['casual'].plot(kind='hist', ax=axes[0])
df['registered'].plot(kind='hist', ax=axes[1])
# TODO: 모수적으로 차이가 좀 있음 이 두 분포에 대한 동질성 검정 가능한지???????? 모수를 같이 가져가도 되는지에 대한 가설.

In [ ]:
df.groupby(['holiday','workingday']).count()

In [ ]:
fig, axes = plt.subplots(1,2) # or ax=fig.add_subplot(221)
df.groupby(['holiday']).sum()['count'].plot(kind='bar', ax=axes[0])
df.groupby(['workingday']).sum()['count'].plot(kind='bar', ax=axes[1])

In [ ]:
df_day_sum.loc[(df_day_sum['holiday']==0) & (df_day_sum['workingday']==0)] 
# holiday도 아니면서 workingday도 아닌 것 = 주말(holiday에도 주말이 있을 수 있으나 단순 주말만 봄)

In [ ]:
df['d'] = 'NA'
df['d'].loc[(df['holiday']==1)] = 'holiday'
df['d'].loc[(df['workingday']==1)] = 'workingday'
df['d'].loc[(df['holiday']==0)&(df['workingday']==0)] = 'weekend'
df['d'].value_counts()

In [ ]:
df2 = pd.concat([df, pd.get_dummies(df['d'], prefix='d')], axis=1)  # categorial -> columns
df2.head()

In [ ]:
df['s'] = 'NA'
df['s'].loc[(df['season']==1)] = 'spring'
df['s'].loc[(df['season']==2)] = 'summer'
df['s'].loc[(df['season']==3)] = 'fall'
df['s'].loc[(df['season']==4)] = 'winter'
df['s'].value_counts()

In [ ]:
# df2 = pd.concat([df2, pd.get_dummies(df['s'], prefix='s')], axis=1)  # season까지 dummy로 추가

In [ ]:
df2.groupby(['d']).sum()['count'].plot(kind='bar')
# 사람들은 주중, 출퇴근 시간에 자전거를 더 많이 탄다

In [ ]:
fig, axes = plt.subplots(1,3)
df.groupby(['hour','d']).sum().unstack()['count']['holiday'].plot(kind='bar', ax=axes[0], color='r', title='holiday')
df.groupby(['hour','d']).sum().unstack()['count']['weekend'].plot(kind='bar', ax=axes[1], color='g', title='weekend')
df.groupby(['hour','d']).sum().unstack()['count']['workingday'].plot(kind='bar', ax=axes[2], color='b', title='workingday')

In [ ]:
df.groupby(['hour','d']).sum().unstack()['count'].plot(kind='bar',color=['r','g','b'])
# 주중은 출퇴근 시간에 많이 이용함이 극명히 보이고
# 주말은 오전 10시 ~ 19시, 증가했다가 감소하는 형태

In [ ]:
df.groupby(['weather','d']).sum().unstack()['count'].plot(kind='bar',color=['r','g','b'])
# 4번일땐 수요가 없다고 봐도 됨. 극명한 결과를 주는 feature로 활용
# 1: Clear, Few clouds, Partly cloudy, Partly cloudy
# 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
# 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
# 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

In [ ]:
df_norm = df.groupby(['hour','d']).sum().reset_index()[['hour','d','casual','registered','count']]

from sklearn.preprocessing import normalize

df_norm['norm_d'] = 'NA'
df_norm['norm_d'].loc[df_norm['d']=='holiday'] = normalize(np.array([df_norm.loc[df_norm['d']=='holiday']['count']]))[0]
df_norm['norm_d'].loc[df_norm['d']=='weekend'] = normalize(np.array([df_norm.loc[df_norm['d']=='weekend']['count']]))[0]
df_norm['norm_d'].loc[df_norm['d']=='workingday'] = normalize(np.array([df_norm.loc[df_norm['d']=='workingday']['count']]))[0]
df_norm['norm_d'] = df_norm['norm_d'].astype(float)
df_norm.groupby(['hour','d']).mean().unstack()['norm_d'].plot(kind='bar',color=['r','g','b'])
# normalize해서 살펴본 결과 출/퇴근 시간은 압도적으로 주중이 많음
# 오히려 새벽 시간 때는 주말이 압도적으로 많음
# 나머지 오후시간(근무시간) 때의 이용은 주중에 이용하는 비율이 현저히 낮음

In [ ]:
fig, axes = plt.subplots(2,1)
df.groupby(['hour','d']).sum().unstack()['casual'].plot(kind='bar',color=['k','y','m'], ax=axes[0])#, stacked=True)
df.groupby(['hour','d']).sum().unstack()['registered'].plot(kind='bar',color=['k','y','m'], ax=axes[1])#, stacked=True)

In [ ]:
fig, axes = plt.subplots(2,1)
df_norm.groupby(['hour','d']).sum().unstack()['casual'].plot(kind='bar',color=['k','y','m'], ax=axes[0])#, stacked=True)
df_norm.groupby(['hour','d']).sum().unstack()['registered'].plot(kind='bar',color=['k','y','m'], ax=axes[1])#, stacked=True)

# 시간대별 일자별 비교를 통해 알수있는 것 = 등록하고 타는 사람들은 정기적으로 출퇴근 시 이용하는 고객이라 볼 수 있음
# 모델링 시 구분할 것 : casual vs registered 따로 모델을 만들어 예측 필요, 시간대별 구분(+날씨변수(계절요인) 등)을 통해 예측 모델 세분화 필요

""" 궁금한것: train에만 있는 feature를 test에 어떻게 활용하지??? 
생각나는것
1. casual, registered 따로 validation으로 예측 후 그 결과를 이용해 count예측..?
"""

In [ ]:
# 날씨에 대해 분석
# 가설: 바람이 많이불고 습하고 기온이 높을 때는 대여에 대한 수요가 낮을 것 같음
sns.heatmap(df[['count','temp','atemp','humidity','windspeed']].corr(), annot=True) # cmap="Greens",

# 습도가 높으면 잘 안타려는 경향이 있는건 맞음, 기온과는 크게 상관은 없는데 이걸 계절별로 끊어보면 달라보일 수 있으니 확인

In [ ]:
from IPython.display import Markdown

In [ ]:
fig = plt.figure()
display(Markdown("<h3>Season</h3>"))
fig.add_subplot(221)
sns.heatmap(df.loc[df['season']==1][['count','temp','atemp','humidity','windspeed']].corr(), annot=True) 
fig.add_subplot(222)
sns.heatmap(df.loc[df['season']==2][['count','temp','atemp','humidity','windspeed']].corr(), annot=True) 
fig.add_subplot(223)
sns.heatmap(df.loc[df['season']==3][['count','temp','atemp','humidity','windspeed']].corr(), annot=True) 
fig.add_subplot(224)
sns.heatmap(df.loc[df['season']==4][['count','temp','atemp','humidity','windspeed']].corr(), annot=True) 
# 여름&가을: 습도가 높으면 자전거를 잘 안타는 경향이 있음 (-0.45이상)

In [ ]:
fig = plt.figure()
display(Markdown("<h3>Season & day_type (summer, fall)</h3>"))
fig.add_subplot(321)
sns.heatmap(df.loc[(df['season']==2)&(df['d']=='holiday')][['count','temp','atemp','humidity','windspeed']].corr(), annot=True) 
fig.add_subplot(323)
sns.heatmap(df.loc[(df['season']==2)&(df['d']=='weekend')][['count','temp','atemp','humidity','windspeed']].corr(), annot=True) 
fig.add_subplot(325)
sns.heatmap(df.loc[(df['season']==2)&(df['d']=='workingday')][['count','temp','atemp','humidity','windspeed']].corr(), annot=True) 
fig.add_subplot(322)
sns.heatmap(df.loc[(df['season']==3)&(df['d']=='holiday')][['count','temp','atemp','humidity','windspeed']].corr(), annot=True) 
fig.add_subplot(324)
sns.heatmap(df.loc[(df['season']==3)&(df['d']=='weekend')][['count','temp','atemp','humidity','windspeed']].corr(), annot=True) 
fig.add_subplot(326)
sns.heatmap(df.loc[(df['season']==3)&(df['d']=='workingday')][['count','temp','atemp','humidity','windspeed']].corr(), annot=True) 
# 습도가 높으면 휴일과 주말에는 자전거 수요가 확 줄어듦, 주중에는 크게 상관없는 것으로 나타남

In [ ]:
# df2 = df.copy(deep=True)
# df2['temp'] = df2['temp'].round()
# df2['atemp'] = df2['atemp'].round()

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.add_subplot(321)
df2.loc[(df2['season']==2)&(df2['d']=='holiday')].groupby(['temp']).sum()['count'].plot(kind='bar')
fig.add_subplot(323)
df2.loc[(df2['season']==2)&(df2['d']=='weekend')].groupby(['temp']).sum()['count'].plot(kind='bar')
fig.add_subplot(325)
df2.loc[(df2['season']==2)&(df2['d']=='workingday')].groupby(['temp']).sum()['count'].plot(kind='bar')
fig.add_subplot(322)
df2.loc[(df2['season']==2)&(df2['d']=='holiday')].groupby(['atemp']).sum()['count'].plot(kind='bar')
fig.add_subplot(324)
df2.loc[(df2['season']==2)&(df2['d']=='weekend')].groupby(['atemp']).sum()['count'].plot(kind='bar')
fig.add_subplot(326)
df2.loc[(df2['season']==2)&(df2['d']=='workingday')].groupby(['atemp']).sum()['count'].plot(kind='bar')
plt.show()
df2.groupby(['season','d']).mean()[['temp','atemp']].plot(kind='bar')
# 사실상 기온과 체감기온은 크게 수요에 영향을 미친다고 보기 어렵고, 두개의 상관도가 매우 유사하므로 온도에 대한 요인을 선택한다면 temp하나만 활용

In [ ]:
display(df2.info())
df2.head()

In [ ]:
# np.log1p()
# np.expm1()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
def get_variance_inflation_factor(X):
    vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif_df = pd.DataFrame({'features': X.columns, 'VIF': vif})
    return vif_df

xx = [x for x in df2._get_numeric_data().columns if x not in ['casual','registered', 'count', 'atemp', 'temp', 'holiday','workingday','weekend']]

get_variance_inflation_factor(df2[xx])

# df.select_dtypes(exclude = ['object'])
# df.select_dtypes(include= np.number)

In [ ]:
# features = ['temp', 'humidity','weather', 'windspeed', 'year', 'month', 'hour', 'd_holiday','d_workingday','d_weekend', 's_spring', 's_summer', 's_fall', 's_winter']
# features = ['season', 'holiday', 'workingday', 'temp', 'humidity','weather', 'windspeed', 'year', 'month', 'hour']
features = ['season', 'temp', 'humidity','weather', 'windspeed', 'year', 'month', 'hour', 'd_holiday','d_workingday','d_weekend']
# features = ['season', 'temp', 'weather', 'year', 'month', 'hour', 'd_holiday','d_workingday','d_weekend']
target = 'count'

In [ ]:
import statsmodels.api as sm
model = sm.OLS.from_formula(f"{target} ~ {'+'.join(features[:-3])}", data=df2)
result = model.fit()
result.summary()

## 예측 모델링
* 구분. holiday vs workingday vs weekend
* feature: season, temp, humidity, windspeed, hour
* (테스트:구분3으로 두거나 요인으로 추가 = season, hour)
* 예측 casual vs registered vs count

* 모델 -> linear regression, random forest regressor 일단 이렇게 두개만 수행

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
import sklearn.metrics 
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df2[features], df2[target], test_size=0.3, random_state=42)

In [ ]:
models = {
    'lr': LinearRegression(),
    'lasso': Lasso(),
    'ridge': Ridge(),
    'rf': RandomForestRegressor(),
    'xgb': XGBRegressor(verbosity=0), # message 출력=1
}
params = {
    'lr':  {},        
    'lasso': { 'lasso__alpha': [1, 10, 100, 1000]},
    'ridge': { 'ridge__alpha': [1, 10, 100, 1000]},
    'rf': {'rf__n_estimators': [3, 10, 30], 'rf__max_features': [2, 4, 6, 8]},
#         {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
    'xgb': {'xgb__n_estimators': [3, 10, 30], 'xgb__max_depth': [3,6], 'xgb__booster': ['gbtree', 'gblinear', 'dart']},
}
# dict_keys(['memory', 'steps', 'scaler', 'xgb', 'scaler__copy', 'scaler__with_mean', 'scaler__with_std', 'xgb__base_score', 'xgb__booster', 
#            'xgb__colsample_bylevel', 'xgb__colsample_bynode', 'xgb__colsample_bytree', 'xgb__gamma', 'xgb__importance_type', 'xgb__learning_rate',
#            'xgb__max_delta_step', 'xgb__max_depth', 'xgb__min_child_weight', 'xgb__missing', 'xgb__n_estimators', 'xgb__n_jobs', 'xgb__nthread', 'xgb__objective', 'xgb__random_state', 
#            'xgb__reg_alpha', 'xgb__reg_lambda', 'xgb__scale_pos_weight', 'xgb__seed', 'xgb__silent', 'xgb__subsample', 'xgb__verbosity'])
pipelines = []
for m,v in models.items():
    pipelines.append(
        Pipeline([
            ('scaler', StandardScaler()),
            (m,v)
        ])
    )
pipelines

In [ ]:
scores = []
best_model = {}
import time
for p in pipelines:
    st_time = time.time()
    m = p.steps[1][0]
    print("----------------------------------------", m.upper(), "----------------------------------------")
#     print(p.get_params().keys())
    p.fit(X_train, y_train)
    y_pred = p.predict(X_train) 
    print("방법1. --")
    print(f"{m} model training score: {p.score(X_train, y_train)}")
    print(f"{m} model validation score: {p.score(X_val, y_val)}")
    print("방법2. --")
    s = cross_val_score(p, X_train, y_train, cv=StratifiedKFold(n_splits=3))
    print(s)
    print("validation score: ", np.mean(s))  # 테스트 cross validation, k-fold=3, stratified
    print("방법3. --")
    grid = GridSearchCV(p, params[m], cv=StratifiedKFold(n_splits=3), n_jobs=-1)
    grid.fit(X_train, y_train)
    print(grid.best_params_, grid.best_score_)
    print(f"duration: {np.round(time.time() - st_time, 3)} s",)    
    best_model.update({m: grid.best_estimator_})
    scores.append(grid.best_score_)
print("==================")
best_model    

In [ ]:
fig, axes = plt.subplots(1,5, figsize=(20,5))
i = 0
for m, p in best_model.items():
    params = p.steps[1][1].get_params()
    try: # Linear Models
        f_impt = p.steps[1][1].coef_
    except:
        f_impt = p.steps[1][1].feature_importances_
    df_f_impt = pd.Series(abs(f_impt), index=features).sort_values(ascending=True)
    df_f_impt.plot(kind='barh', ax=axes[i])
    i+=1    

In [ ]:
pd.DataFrame(scores, list(models.keys())).plot(kind='bar', rot=0, alpha=0.75)

In [ ]:
df_test['d'] = 'NA'
df_test['d'].loc[(df_test['holiday']==1)] = 'holiday'
df_test['d'].loc[(df_test['workingday']==1)] = 'workingday'
df_test['d'].loc[(df_test['holiday']==0)&(df_test['workingday']==0)] = 'weekend'
df_test['d'].value_counts()
# df_test['s'] = 'NA'
# df_test['s'].loc[(df_test['season']==1)] = 'spring'
# df_test['s'].loc[(df_test['season']==2)] = 'summer'
# df_test['s'].loc[(df_test['season']==3)] = 'fall'
# df_test['s'].loc[(df_test['season']==4)] = 'winter'
# df_test['s'].value_counts()

In [ ]:
df_test2 = pd.concat([df_test, pd.get_dummies(df_test['d'], prefix='d')], axis=1)
df_test2.head()

In [ ]:
# df_test2 = pd.concat([df_test2, pd.get_dummies(df_test2['s'], prefix='s')], axis=1)
# df_test2.head()

### 최종 결과 제출

In [ ]:
print(features)
estimator = best_model.get('xgb')
y_pred = estimator.predict(df_test2[features])
res_df = pd.concat([df_test2['datetime'], pd.DataFrame({'count': abs(y_pred)})], axis=1)
res_df.head()

In [ ]:
res_df.to_csv("submission.csv", index=False) 

In [ ]:
# 0. features = ['season', 'holiday', 'workingday', 'temp', 'humidity', 'weather', 'windspeed', 'year', 'month', 'hour']
# RF   : 0.48799
# XGB: 0.47318

# 1. features = ['season', 'temp', 'humidity','weather', 'windspeed', 'year', 'month', 'hour', 'd_holiday','d_workingday','d_weekend']
res_df.to_csv("submission1.csv", index=False)  # Score: 0.49349 (round안할때),  0.49386 (round할때)
res_df.to_csv("submission2.csv", index=False)  # XGB: 0.46480 (round안할때), 0.46568 (round할때)

# 2. features = ['temp', 'humidity','weather', 'windspeed', 'year', 'month', 'hour', 'd_holiday','d_workingday','d_weekend']
# 0.50444 0.5가 넘어버려서 모두 제외 
# 3. features = ['temp', 'humidity','weather', 'windspeed', 'year', 'month', 'hour', 'd_holiday','d_workingday','d_weekend', 's_spring', 's_summer', 's_fall', 's_winter']
# 0.50210 (RF), 0.50318 (XGB)

# humidity outlier detection 
# RF   : 0.48681
# XGB: 0.48484

0.48142



| trial | features | RF | XGB | 
| :---| :---| :---| :---|
| 1 | ['season', 'holiday', 'workingday', 'temp', 'humidity', 'weather', 'windspeed', 'year', 'month', 'hour'] | 0.48799 | 0.47318 |
| 2 | ['season', 'temp', 'humidity','weather', 'windspeed', 'year', 'month', 'hour', 'd_holiday','d_workingday','d_weekend'] | 0.49349 | <span style='color:red'>**0.46480**</span> |
| 3 | ['temp', 'humidity','weather', 'windspeed', 'year', 'month', 'hour', 'd_holiday','d_workingday','d_weekend'] | 0.50444 | 0.50XXX |
| 4 | ['temp', 'humidity','weather', 'windspeed', 'year', 'month', 'hour', 'd_holiday','d_workingday','d_weekend', 's_spring', 's_summer', 's_fall', 's_winter'] | 0.50210 | 0.50318 |
| 5 | 2번 + humidity outlier detection  | 0.48681 | 0.48484 |

parameter xgb: {} -> 0.48142

In [ ]:
# df = pd.DataFrame()
# for d,v in df_test.groupby('day_type'):
#     print(d, "----------")
#     v = v[features].drop(columns='day_type')
#     print(v.columns)
#     print(best_model[d])
#     y_pred = np.round(best_model[d].predict(v))
#     print(y_pred)
#     datetime_id = df_test.loc[v.index,'datetime']
# #     display(get_variance_inflation_factor(v[features]))
#     df = pd.concat([df, pd.concat([datetime_id.reset_index(drop=True), 
#                                    pd.DataFrame({'count':y_pred})],axis=1)])
# df